In [1]:
from langchain.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(r"documents\2101.01158v1.pdf")
docs = loader.load()


In [2]:
for i, doc in enumerate(docs[:3]):
    print(f"Doc {i} Preview:\n", doc.page_content[:500])


Doc 0 Preview:
 A Hybrid Learner for Simultaneous Localization
and Mapping
Thangarajah Akilan, Member, IEEE, Edna Johnson, Japneet Sandhu, Ritika Chadha, Gaurav Taluja
Abstract—Simultaneous localization and mapping (SLAM) is
used to predict the dynamic motion path of a moving platform
based on the location coordinates and the precise mapping of the
physical environment. SLAM has great potential in augmented
reality (AR), autonomous vehicles, viz. self-driving cars, drones,
Autonomous navigation robots (ANR). Th
Doc 1 Preview:
 consistent map of this unknown environment and determine
the location relative to the map. Through SLAM, robots and
vehicles can be truly and completely automated without any
or minimal human intervention. But the estimation of maps
consists of various other entities, such as large storage issues,
precise location coordinates, which makes SLAM a rather
intriguing task, especially in the real-time domain.
Many researches have been done worldwide to determine
the e

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunked_docs = splitter.split_documents(docs)


In [4]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

db = Chroma.from_documents(
    documents=chunked_docs,
    embedding=embedding_model,
    persist_directory="chroma_db"
)


In [6]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_ollama import ChatOllama

# Connect to your local LLaMA model
llm = ChatOllama(model="llama3.1:8b")

# Setup conversational chain with memory
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=db.as_retriever(),
    memory=ConversationBufferMemory(return_messages=True)
)


In [10]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=db.as_retriever(),
    memory=memory
)

chat_history = []  # Can ignore this, memory handles it internally

print("🧠 AI Assistant ready. Type 'exit' to quit.")

while True:
    query = input("\nYou: ")
    if query.lower() in ("exit", "quit"):
        break

    # Only pass the question
    result = qa_chain.invoke({"question": query})

    print("\nAssistant:", result["answer"])


🧠 AI Assistant ready. Type 'exit' to quit.

Assistant: Based on the provided context, I can describe SLAM (Simultaneous Localization and Mapping) as follows:

SLAM is an algorithm that computes the relative location of a moving platform or vehicle on a pre-defined map. It has various applications in fields such as self-driving vehicles, space and maritime exploration, indoor positioning, and search and rescue operations.

The primary responsibility of a SLAM algorithm is to produce an understanding of the environment and the location of the vehicle by providing its coordinates. This enables the formation of a trajectory to determine the view at a particular point in time.

In essence, SLAM algorithms aim to simultaneously localize (determine the position) and map (build a representation of the environment) the surrounding area, often using sensor data such as cameras, lidar, or GPS.

Assistant: According to the provided context, PoseNet is mentioned as a neural network that can be used

KeyboardInterrupt: Interrupted by user